# Intro to Spark

**Disclaimer:** TP plus compliqué !

**Spark** est un système de calcul hautement parallélisé :
- au niveau du stockage : la data est fragmentée, dupliquée, répartie sur un nombre quelconque de disques/servers
- au niveau du calcul : les calculs s'exécutent en parallèle sur plusieurs machines, chacune sur sa portion de data

Spark en tant que tel est un framework qui a des implémentation dans plusieurs langages :
- Python via PySpark (ce qe nous utiliserons)
- Scala
- R
- Java

## Quelques informations en vrac sur Spark
### Hardware
- Spark peut s'installer sur un grand nombre de machines situées dans un même réseau. Elles pourront ensuite se reconnaître et collaborer. 1 unité de calcul = 1 noeud.
- Spark fonctionne sur le mode master/worker : un noeud est désigné `master` et jouera le rôle de chef d'orchestre pour que les autres noeuds `workers` exécutent les tâches dans le bon ordre
- Les noeuds Spark communiquent énormément entre eux pour s'échanger des informations et surtout des données

### Software
- Un code Spark / PySpark doit utiliser les primitives Spark pour que tout s'exécute selon la logique Spark
- Le code pyspark est transmis au noeud `master` qui le lit et prépare l'orchestration des calculs selon les `workers` qu'il a à disposition. Seuls les `workers` manipuleront la donnée (sauf exception)
- Spark est *lazy* : `master` ne lance réellement aucun calcul tant qu'il n'a pas lu d'opération impliquant l'affichage ou l'écriture des résultats
- Corrolaire du *lazy* : sans précaution, Spark peut répéter plusieurs fois les mêmes calculs ... Exemple avec 2 chaînes de transformation data `A -> B -> C -> D` suivi de `A -> B -> E`. Les étapes intermédiaires `A -> B` sont identiques mais pour calculer `D` et `E`, Spark risque de les exécuter 2 fois. Apprendre à manipuler les méthodes [cache](https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.cache.html?highlight=cache) !

### Framework
- Spark se base sur des `DataFrame` très proches en terme d'utilisation des `pandas.DataFrame` donc pas de panique :)
- Spark gère tout ce que sait gérer SQL mais part de fichiers plats (ici CSV) : join, select, etc ...
- PySpark permet de gérer tout ces concepts avec la facilité d'accès du Python
- PySpark est TRÈS typé et a besoin de connaître les types de chaque colonne manipulées
- Spark est TRÈS flexible en terme de configuration et d'exécution et cela peut sembler déroutant pour des exemples simples

### Organisation du calcul
En informatique, à chaque architecture ses optimisations :
- en local sur 1 CPU, un calcul possède peu d'optimisation : simple et linéaire, possiblement async
- en local sur multi CPU, un calcul doit être prévu pour paralléliser les exécutions sur plusieurs coeurs physiques
- en local sur mono/multi GPU, un calcul doit être hautement parallélisable, découpable en tranche de data qui tiennent en GPU-RAM
- en multi machine multi GPU, idem que plus haut avec bande passante importante entre machine pour échange d'information

... Spark gère le multi machine, multi CPU, multi RAM, multi disque : calculs hautement parallélisés grâce à la magie de Spark, data échangées au mieux entre machine (possiblement avec l'aide humaine).
__Spark est toujours prêt à gérer un contexte d'exécution très complexe__ => il faut s'attendre à beaucoup d'overhead sur des cas simples

**Les opérations s'exécutent sur des workers séparés, en parallèle**, il faut donc parfois faire "un peu attention" à la façon dont on demande à Spark de *partitionner* sa data.

## À retenir

1. Spark et son implémentation PySpark sont très puissant car gèrent un parallélisme quasi infini et réglable à 100%
2. PySpark a un coût d'entrée pour se couler dans le moule Spark mais permet de réaliser des opérations très complexes avec la simplicité du Python
3. Votre notebook n'exécutera n'a pas accès à la data manipulée et n'effectuera aucun calculs ; il les transmettra au Spark Master qui les répartira entre ses workers qui ont accès à la data

## Exemple de code Spark

In [5]:
import langdetect

In [6]:
import requests

def get_embedding(text: str):
    url = "http://embedder:8000/embed"
    payload = {"text": text}
    
    response = requests.post(url, json=payload)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

In [78]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [79]:
MY_ID = "François"

In [80]:
# Initialize PySpark session
assert MY_ID is not None, "provide your id first!"
spark = SparkSession.builder \
    .appName(MY_ID) \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "4g") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "2") \
    .getOrCreate() 

# /!\ Tout se fera à partir de cet object magique `spark`

25/03/12 14:52:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [102]:
%%time
data = [("Alice", 1), ("Bob", 2), ("Catherine", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])

# Show the dataframe
df.show()

[Stage 42:>                                                         (0 + 1) / 1]

+---------+-----+
|     Name|Value|
+---------+-----+
|    Alice|    1|
|      Bob|    2|
|Catherine|    3|
+---------+-----+

CPU times: user 0 ns, sys: 8.07 ms, total: 8.07 ms
Wall time: 1.16 s


In [84]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, FloatType, ArrayType, StructField, StructType, DoubleType

In [85]:
df_beers = spark.read.csv("/datasets/csv/beers.csv", header=True)
df_beers.show()

+---+----------+--------------------+------+--------+---+---+---+---+--------+--------------------+--------+-------------------+
| id|brewery_id|                name|cat_id|style_id|abv|ibu|srm|upc|filepath|            descript|add_user|           last_mod|
+---+----------+--------------------+------+--------+---+---+---+---+--------+--------------------+--------+-------------------+
|  1|       812|         Hocus Pocus|    11|     116|4.5|  0|  0|  0|    NULL|Our take on a cla...|       0|2010-07-22 20:00:20|
|  2|       264|   Grimbergen Blonde|    -1|      -1|6.7|  0|  0|  0|    NULL|                NULL|       0|2010-07-22 20:00:20|
|  3|       779|Widdershins Barle...|    -1|      -1|9.1|  0|  0|  0|    NULL|                NULL|       0|2010-07-22 20:00:20|
|  4|       287|             Lucifer|    -1|      -1|8.5|  0|  0|  0|    NULL|                NULL|       0|2010-07-22 20:00:20|
|  5|      1056|              Bitter|    -1|      -1|  4|  0|  0|  0|    NULL|                NUL

In [86]:
%%time
# Define a specific funtion to map beer names
@F.udf(returnType=StringType())
def revert_cap_name(name: str):
    return name[::-1].upper()


df_beers.limit(10).withColumn("reverse_capitalized", revert_cap_name(F.col("name"))).select("name", "reverse_capitalized").show()

[Stage 2:>                                                          (0 + 1) / 1]

+--------------------+--------------------+
|                name| reverse_capitalized|
+--------------------+--------------------+
|         Hocus Pocus|         SUCOP SUCOH|
|   Grimbergen Blonde|   EDNOLB NEGREBMIRG|
|Widdershins Barle...|ENIWYELRAB SNIHSR...|
|             Lucifer|             REFICUL|
|              Bitter|              RETTIB|
|       Winter Warmer|       REMRAW RETNIW|
|Winter Welcome 20...|8002-7002 EMOCLEW...|
|       Oatmeal Stout|       TUOTS LAEMTAO|
|     Espresso Porter|     RETROP OSSERPSE|
|     Chocolate Stout|     TUOTS ETALOCOHC|
+--------------------+--------------------+

CPU times: user 8.17 ms, sys: 375 μs, total: 8.54 ms
Wall time: 1.14 s


## Observations
Que remarque-t-on tout de suite ?

# Uses cases

# UC-1 : description data

- Q1: Combien y a-t-il de bières dans la DB ?
- Q2: Top10 brasseries les plus représentées avec le nombre de bière par brasserie ?
- Q3: Top10 des bières les plus fortes (ABV) en France ?
- Q4: Par pays, nombre de brasseries qui proposent des bières de type `Porter` et ABV moyen de celles-ci ?
- Q5: Mediane du nombre de bière par pays ?

In [87]:
%%time
# Q1
df_beers = spark.read.csv("/datasets/csv/beers.csv", header=True)
df_beers.count()

CPU times: user 3.33 ms, sys: 0 ns, total: 3.33 ms
Wall time: 508 ms


7059

In [88]:
df_beers = (
    df_beers
    .withColumnRenamed("abv", "abv_old")
    .withColumn("abv", F.col("abv_old").cast(FloatType()))
    .drop("abv_old")
)
df_beers.count()

7059

In [89]:
df_brew = spark.read.csv("/datasets/csv/breweries.csv", header=True)

In [90]:
%%time
#Q2 
df_brew = spark.read.csv("/datasets/csv/breweries.csv", header=True)
df = (
    df_beers
    .join(df_brew, on=df_beers.brewery_id==df_brew.id)
    .groupby([df_brew.id, df_brew.name])
    .count()
    .sort(F.col("count").desc())
)
df.show()

+----+--------------------+-----+
|  id|                name|count|
+----+--------------------+-----+
| 858|Midnight Sun Brew...|   57|
|1072|          Rogue Ales|   49|
|  44|      Anheuser-Busch|   38|
| 483|        Egan Brewing|   37|
|1286|      Troegs Brewing|   37|
| 157| Boston Beer Company|   36|
|1268|   Titletown Brewing|   34|
| 512|   F.X. Matt Brewing|   34|
|1142|Sierra Nevada Bre...|   33|
|1204|   Stone Brewing Co.|   32|
|1173|Southern Tier Bre...|   31|
|1352|Weyerbacher Brewi...|   30|
| 590|   Gottberg Brew Pub|   28|
| 585|Goose Island Beer...|   28|
|  45|Appalachian Brewi...|   27|
| 459|Dogfish Head Craf...|   27|
|1326|     Victory Brewing|   25|
| 100| Bell's Brewery Inc.|   25|
| 441|   Deschutes Brewery|   22|
| 960|Otto's Pub and Br...|   21|
+----+--------------------+-----+
only showing top 20 rows

CPU times: user 6.08 ms, sys: 173 μs, total: 6.26 ms
Wall time: 936 ms


In [91]:
# Q3 top 10 bières FR les plus fortes
df_beers_and_brew = df_beers.join(df_brew, on=df_beers.brewery_id==df_brew.id).cache()
df_beers_and_brew.count()

5906

In [92]:
df = (
    df_beers_and_brew
    .filter(df_beers_and_brew.country==F.lit("France"))
    .sort(F.col("abv").desc())
    .select([df_beers.name, "abv"])
)
df.show()

+--------------------+----+
|                name| abv|
+--------------------+----+
|           Belzebuth|13.0|
|Gavroche French R...| 8.5|
|             3 Monts| 8.5|
|                Yeti| 8.0|
|      Jenlain Blonde| 7.5|
|              Blonde| 7.5|
|   Les Sans Culottes| 7.0|
|           Framboise| 6.0|
|Jenlain St Druon ...| 6.0|
|Castelain St.Aman...| 5.9|
|Castelain Blond B...| 5.6|
|                XTra| 4.5|
|                Jade| 4.5|
|     BiÃ¨re de NoÃ«l| 0.0|
|AmbrÃ©e / Chestnu...| 0.0|
|    BÃªte des Vosges| 0.0|
|                1664| 0.0|
|                Ambr| 0.0|
+--------------------+----+



In [93]:
a[a.style_name.apply(lambda s: "porter" in s.lower())]

,id,cat_id,style_name,last_mod
17,18,1,Robust Porter,2010-06-15 19:17:37
18,19,1,Brown Porter,2010-06-15 19:17:43
24,25,2,Porter,2010-06-15 19:21:39
38,39,3,Smoke Porter,2010-06-15 19:24:30
44,45,3,American-Style Imperial Porter,2010-06-15 19:25:33
45,46,3,Porter,2010-06-15 19:25:53
105,106,9,Baltic-Style Porter,2010-06-15 19:42:40


In [94]:
%%time
print("Q4")
df_style = spark.read.csv("/datasets/csv/styles.csv", header=True)
target_style_id = (
    df_style
    .filter(F.col("style_name").ilike("porter"))
    .select(F.col("id").alias("style_id"))
)
dd = (
    df_beers_and_brew
    .join(target_style_id, how="inner", on="style_id")
    .select([df_brew.name, df_brew.name.alias("brewer_name"), "abv", "country"])
    .groupby("country")
    .agg(F.avg("abv").alias("avg_abv"), F.countDistinct("brewer_name").alias("n_brewer_having_porter"))
    .sort(F.col("n_brewer_having_porter").desc())
)
dd.show()

Q4
+--------------+------------------+----------------------+
|       country|           avg_abv|n_brewer_having_porter|
+--------------+------------------+----------------------+
| United States|2.3305555541291194|                   216|
|United Kingdom|3.9500000136239186|                    13|
|        Canada|               0.0|                     5|
|        Russia|               7.0|                     1|
|        Sweden|               5.5|                     1|
|       Germany| 7.099999904632568|                     1|
|     Lithuania| 6.800000190734863|                     1|
|        Norway|               7.0|                     1|
|       Denmark|               8.0|                     1|
|   Switzerland|               4.5|                     1|
|Czech Republic|               8.0|                     1|
|         Japan|               0.0|                     1|
|        Poland| 8.300000190734863|                     1|
|     Australia|               0.0|                  

In [95]:
%%time
print("Q5")
dd = (
    df_beers_and_brew
    .groupby("country")
    .count()
    #.agg(F.median("count"))
)
#print("Q5:", dd.first()[0])
dd.show()

Q5
+--------------------+-----+
|             country|count|
+--------------------+-----+
|              Russia|    6|
|Macedonia, the Fo...|    1|
|               Macao|    1|
|              Sweden|    7|
|         Philippines|    2|
|             Germany|  302|
|              France|   18|
|              Greece|    2|
|           Sri Lanka|    2|
|                Togo|    1|
|            Slovakia|    1|
|           Argentina|    4|
|             Belgium|  331|
|             Finland|    3|
|             Myanmar|    1|
|        Sierra Leone|    1|
|       United States| 4552|
|               India|   15|
|               China|    2|
|             Croatia|    1|
+--------------------+-----+
only showing top 20 rows

CPU times: user 3.7 ms, sys: 0 ns, total: 3.7 ms
Wall time: 214 ms


In [96]:
dd.toPandas()

,country,count
0,Russia,6
1,"Macedonia, the Former Yugoslav Republic of",1
2,Macao,1
3,Sweden,7
4,Philippines,2
...,...,...
56,Hungary,4
57,Mauritius,1
58,United Kingdom,210
59,"Korea, Republic of",3


# UC-2 : préparer un dataset de ranking 
Tout moteur de recherche/search-engine - **SE** - nécessite de la configuration ... beaucoup de configuration. Une des configuration très orientée "data" est le calcul que l'index doit opérer pour scorer chaque réponse possible face à une requête. L'apprentissage statistique de ce score s'appelle *Learning to Rank*  - **LTR** - et nécessite des connaissances poussées en machine learning. 

Cette tâche LTR se base sur les *feedbacks implicites* des utilisateurs face au moteur de recherche. Commençons par un exemple. Quand vous cherchez un objet sur LeBonCoin, vous laissez plusieurs informations *implicites* sur votre perception des résultats proposés : les item sur lesquels vous avez cliqués bien sûr mais également ceux que vous avez probablement *vu* sans cliquer dessus ... Ces "vues sans clics" sont une précieuse information implicite sur les jugement que vous avez porté aux résultats proposés. Pour ce TP nous nous limiterons à ce concept de "vu x click" mais il est possible d'aller plus loin (dwell-time, hierarchisation des interactions explicites, ...). 

On appelle *Search Engine Results Page* - **SERP** - la liste des résultats classés par un SE. Un document qui figure dans les résulats d'une recherche a donc une position (son rang) au sein de la **SERP**.

Exemple, où :
- `query` est la recherche réalisée par un user et qui a débouché sur une SERP
- `clicked_id` : l'id de la bière cliquée par le user
- `user_id` l'id de l'utilisateur (simplifions en disant que c'est même l'id d'une recherche) : permet de retrouver tous les résultats proposés dans **une** recherche
- `id_in_serp` : l'id d'une bière figurant dans la SERP
- `pos_in_serp` : la position/le rang de la bière `id_in_serp` dans la SERP issue de la recherche 

In [97]:
df_pref = spark.read.csv("/datasets/beers_feedback.csv", header=True, inferSchema=True)
df_pref.limit(3).show()

[Stage 37:>                                                         (0 + 1) / 1]

+-----------+----------+--------------------+----------+-----------+
|      query|clicked_id|             user_id|id_in_serp|pos_in_serp|
+-----------+----------+--------------------+----------+-----------+
|fruity sour|      4442|ecfce536-7fc5-11e...|      4442|          1|
|fruity sour|      4442|ecfce536-7fc5-11e...|       475|          2|
|fruity sour|      4442|ecfce536-7fc5-11e...|       481|          3|
+-----------+----------+--------------------+----------+-----------+



Un travail préliminaire au LTR est la constitution d'un dataset qui permet d'aggréger ces feedbacks laissés par tous les utilisateurs ayant réalisé la même query. Chacun a vu et cliqué selon ses propres impressions de pertinence et il convient de "moyenner" tout cela pour obtenir des appréciations globales. L'objectif d'un tel dataset est de pouvoir lister des exemples de triplets `(query, document, note)` qui permet de savoir que face à une *query* `milky stout low bitterness`, un *document* `Super bitter beer brewed with organic roasted barley and chocolate` aura une pertinence de *1/4* (arbitraire). 

Implémenter le modèle d'agrégation de feedback "cascade model" [1] (pour la culture, **inutile d'avoir lu l'article** pour le TD) qui propose une approche pragmatique pour obtenir ces données. La méthode est la suivante :
- pour chaque recherche utilisateur:
    - étudier la position de l'id cliqué dans la SERP - soit `clicked_pos_in_serp` cette information
    - Considérer que tout doc situés "au-dessus dans la SERP" (càd quand `pos_in_serp <= clicked_pos_in_serp`) avait été vu par l'utilisateur
    - Récapituler tous ces documents "vus et cliqués" et "vus mais pas cliqués"
- Pour chaque recherche et bière cliquée (`clicked_id`), calculer la "probabilité de clic sachant qu'elle a été vue", càd le nombre de fois qu'elle a été cliquée divisé par le nombre de fois où elle a été vue


[1] https://dl.acm.org/doi/abs/10.1145/1341531.1341545

In [106]:
# output schema : https://stackoverflow.com/a/54771215/10716281

from pyspark.sql.types import *

mapping = {
    "float64": DoubleType,
    "object":StringType,
    "int64":IntegerType,
    "int32":IntegerType,
    "bool": BooleanType,
} # Incomplete - extend with your types.

def createUDFSchemaFromPandas(dfp):
  column_types  = [StructField(key, mapping[str(dfp.dtypes[key])]()) for key in dfp.columns]
  schema = StructType(column_types)
  return schema

In [107]:
def compute_cascade_model_per_user_query(df: pd.DataFrame) -> pd.DataFrame:
    pos_of_clicked_id = df[df["id_in_serp"] == df["clicked_id"]].iloc[0]["pos_in_serp"]
    df["seen"] = (df["pos_in_serp"] <= pos_of_clicked_id).astype(int)
    df["clicked"] = np.where(df["pos_in_serp"] == pos_of_clicked_id, 1, 0)
    return df

df_processed = compute_cascade_model_per_user_query(df_pref.limit(3).toPandas())
schema = createUDFSchemaFromPandas(df_processed)

In [110]:
dd = (
    df_pref
    .repartition(16, "query", "user_id")
    .groupby(["query", "user_id"])
    .applyInPandas(compute_cascade_model_per_user_query, schema)
    #.filter(F.col("seen") == F.lit(1))
    .groupby(["query", "id_in_serp"])
    .agg(F.sum("seen").alias("n_seen"), F.sum("clicked").alias("n_clicked"))
    .withColumn("clic_proba", F.col("n_clicked") / F.col("n_seen"))
    .filter(F.col("n_seen") >= F.lit(10))
    .sort(F.col("clic_proba").desc())
    .select(["query", "id_in_serp", "clic_proba", "n_seen"])
)
dd.show()

[Stage 62:==========================================>             (12 + 4) / 16]

+--------------------+----------+-------------------+------+
|               query|id_in_serp|         clic_proba|n_seen|
+--------------------+----------+-------------------+------+
|high ABV imperial...|      5778|                0.6|    10|
|amber ale with ca...|      4945|                0.6|    10|
|chocolate stout w...|        10|                0.6|    10|
|non-alcoholic chr...|      2216|                0.5|    12|
|citrusy West Coas...|      5804|                0.5|    10|
|strong sour with ...|      4442|                0.5|    10|
|tropical flavored...|      2922|0.46153846153846156|    13|
|     malty amber ale|      3740|0.45454545454545453|    11|
|         fruity sour|       481| 0.4166666666666667|    12|
|           dry stout|       345|                0.4|    10|
|    malty scotch ale|      5783|0.35714285714285715|    14|
|     smoky rauchbier|      3364| 0.3333333333333333|    15|
|         milky stout|      2001| 0.3333333333333333|    12|
|         bitter sour|  

In [112]:
import langdetect

# UC-3 : récupérer les docs qui parlent d'un mot

Peut-on utiliser SQL pour réaliser un mini moteur de recherche ? Pour différentes requêtes (`query` en anglais) textuelles très simples à base de mot-clef, retrouver les bières qui semblent répondre à la demande. Exemples :
- trouver les bières ou les brasseries qui parlent de bières "fine"
- idem pour "juicy"
- idem pour "genuine"
- idem pour les bières mâturées dans des "oak cask" (fûts en chêne) -> combien y en a-t-il ? $N_1$
   - idem pour les bières qui évoquent uniquement "cask" -> combien y en a-t-il ? $N_{1,1}$
   - idem pour celles ne parlant que de "oak" -> combien y en a-t-il ? $N_{1,2}$
- idem pour les bières qui évoquent "oak" et "cask" -> combien y en a-t-il ? $N_{2}$

In [ ]:
%%time

## UC-3.1 langdetect over descripts

In [132]:
import langdetect

@F.udf(returnType=StringType())
def detect_lang(text):
    try:
        return langdetect.detect(text)
    except langdetect.LangDetectException:
        return "ukn"

In [128]:
@F.udf(returnType=StringType())
def agg_descriptions(beer_descr, brewer_descr):
    descr = ""
    if beer_descr:
        descr += " " + beer_descr
    if brewer_descr:
        descr += " " + brewer_descr
    return descr

In [134]:
dd = (
    df_beers.withColumnRenamed("descript", "beer_descr")
    .join(
        df_brew.withColumnRenamed("descript", "brewer_descr"),
        on=df_beers.brewery_id==df_brew.id
    )
    .withColumn(
        "descr", 
        agg_descriptions(
            F.coalesce(F.col("beer_descr"), F.lit("")), 
            F.coalesce(F.col("brewer_descr"), F.lit(""))
        )
    )
    .repartition(4)
    .withColumn("lang", detect_lang(F.col("descr")))
    .groupby(F.col("lang")).count()
).show()

[Stage 86:=============================>                            (2 + 2) / 4]

+----+-----+
|lang|count|
+----+-----+
|  en| 2354|
|  vi|    4|
|  de|    6|
|  es|    1|
|  af|   62|
| ukn| 3464|
|  fr|    3|
|  tl|    3|
|  nl|    2|
|  cy|    1|
|  ro|    1|
|  hu|    1|
|  ca|    2|
|  it|    1|
|  da|    1|
+----+-----+



In [ ]:
# your code
%%time

# UC-4 : vectorisation des description des bières
Préparer le recours à un service de vectorisation qui permettra de convertir la connaissance sur une bière en un vecteur numérique. Ce vecteur permet de sythétiser mathématiquement l'information disponible sur une bière et sa brasserie et pourra être réutilisé plus tard dans un moteur de recherche.
à faire :
- Préparer une description la plus complète possible pour chaque bière
- envoyer ces descriptions une à une via un appel HTTP sur `vectorizer` (voir instruction plus bas)


## Service de vectorisation
Nous allons faire appel à un service de vectorisation dans le cluster http://vectorizer. Une autre option aurait été d'utiliser un service de vectorisation externe - Jina - qui propose gratuitement 1M token de vectorisation ... mais problème de black listing de l'IP de l'UL.

Snippet de code pour l'utiliser :

In [ ]:
import requests

def get_embedding(text: str):
    url = "http://vectorizer:8000/embed"
    payload = {"text": text}
    
    response = requests.post(url, json=payload)
    try:
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()["vector"]
    except:
        return []


Proposer un texte synthétique à vectoriser, puis paralléliser les appels au Vectorizer via Spark:

In [ ]:
# your code

# UC-5 : answer question in corpa

**Question difficile en Spark**

**Grandes lignes :** trouvons les documents qui répondent à une question. Exemple : à partir de la description vectorisée à UC-4 pour chaque bière, comment trouver les bières qui répondent à une description plus complète ? Exemple:
- "very bitter beer with smoky taste"
- "fruity sour - balanced sourness"
- "weird beer"

Voir la doc [Spark ML lib - feature extraction](https://spark.apache.org/docs/latest/api/python/reference/pyspark.mllib.html#feature) pour trouver des idées (TF-IDF, Word2Vec), ou utiliser le résultats de vos vectorisation de UC-4.

In [ ]:
queries = ["very bitter beer with smoky taste", "fruity sour - balanced sourness", "weird beer"]